In [6]:
%pip install pydantic openai

  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.41.5-cp314-cp314-macosx_11_0_arm64.whl.metadata (7.3 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.12.0-cp314-cp314-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached pydantic-2.12.5-py3-none-any.whl (463 kB)
Using cached pydantic_core-2.41.5-cp314-cp314-macosx_11_0_arm64.whl (1.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.5 MB/s  0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached jiter-0.12.0-cp314-cp314-macosx_11_0_arm64.whl (318 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

In [4]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


# Using Tools
This notebook demonstrates how we can use tools to gather information and provide more accurate responses. We'll build a weather-checking mock as an example.

## What we'll learn:
- Basic interaction with Language Models (LLM)
- How to create and use tools with AI
- The complete flow of an AI agent using tools
- Understanding the message flow in a tool-enabled conversation

### Setup

In [16]:
import json
from dotenv import load_dotenv
from lib.messages import UserMessage, SystemMessage, ToolMessage  # Different message types
from lib.tooling import tool  # Tool decorator for creating AI tools
from lib.llm import LLM  # Our Language Model wrapper
import os

In [17]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [18]:
import os
print("OPENAI_API_KEY starts with:", os.getenv("OPENAI_API_KEY")[:10])

OPENAI_API_KEY starts with: sk-proj-vM


In [19]:
chat_model = LLM(
    
)

## Basic LLM Interaction
Before we dive into tools, let's understand how to interact with our Language Model in its simplest form. 
There are two main ways to communicate with the model:

In [15]:
# Method 1: Simple single-turn query
response = chat_model.invoke("What is an AI Agent?")
print("Single Query Response:\n", response.content)

Single Query Response:
 An AI agent is a system or entity that uses artificial intelligence techniques to perceive its environment, make decisions, and take actions to achieve specific goals. AI agents can operate autonomously or semi-autonomously and can be found in various applications, ranging from simple rule-based systems to complex machine learning models.

Key characteristics of AI agents include:

1. **Perception**: AI agents can gather information from their environment through sensors or data inputs. This could involve processing visual data, audio signals, or other forms of input.

2. **Decision-Making**: Based on the information they perceive, AI agents can analyze the data and make decisions. This can involve reasoning, planning, and learning from past experiences.

3. **Action**: After making decisions, AI agents can take actions to interact with their environment. This could involve physical actions (like a robot moving) or digital actions (like a software agent executin

In [20]:
# Method 2: Multi-turn conversation with specific roles
messages = [
    SystemMessage(content="You're an OpenAI API specialist"),
    UserMessage(content="What is Function Calling?")
]
response = chat_model.invoke(messages)
print("\nStructured Conversation Response:\n", response.content)


Structured Conversation Response:
 Function calling is a programming concept where a specific function (a block of code designed to perform a particular task) is executed or invoked within a program. When a function is called, the program temporarily transfers control to that function, executes its code, and then returns control back to the point where the function was called. 

### Key Concepts of Function Calling:

1. **Function Definition**: Before a function can be called, it must be defined. This includes specifying the function's name, parameters (if any), and the code that will be executed.

2. **Parameters and Arguments**: Functions can take inputs known as parameters. When calling a function, you provide arguments that correspond to these parameters.

3. **Return Values**: Functions can return values to the caller. This allows the function to send back results after performing its task.

4. **Scope**: Variables defined within a function are typically local to that function an

## Building an AI Tool
Now let's make our AI more capable by giving it a tool to check the weather. 
This demonstrates how we can extend AI capabilities beyond just conversation.

### Understanding the Tool Structure:
1. We use the `@tool` decorator to mark a function as an AI tool
2. The tool needs clear documentation and typed parameters
3. The tool should return structured data

In [7]:
@tool
def get_weather(city: str):
    """Get the current temperature for a city.
    
    Args:
        city (str): Name of the city to check weather for
        
    Returns:
        dict: Contains temperature information for the requested city
    """
    # In a real application, this would call a weather API
    mock_weather = {
        "São Paulo": "28°C",
        "Oslo": "-3°C",
        "New York": "15°C",
        "Tokyo": "22°C"
    }
    return {"temperature": mock_weather.get(city, "Unknown")}

In [8]:
# Bind the tool to an LLM
chat_model_with_tools = LLM(tools=[get_weather])

## Understanding the Tool Usage Flow
Let's break down how the AI uses tools step by step:

1. User asks a question about weather
2. AI recognizes the need to use the weather tool
3. AI makes a tool call
4. Tool executes and returns results
5. AI processes the tool's response
6. AI formulates a natural language response

Let's see this in action:

In [12]:
# Set up our system with clear instructions
messages = [
    SystemMessage(
        content="You are a helpful assistant that can access a tool to get current temperature " 
                "for cities. Use the tool whenever someone asks about the weather or temperature " 
                "in a specific location. Infor the user if you don't know the answer."
    ),
    UserMessage(content="How cold is it in Oslo?")
]

In [13]:
# AI decides to use the weather tool
ai_message = chat_model_with_tools.invoke(messages)
ai_message

AIMessage(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(id='call_LdCIo0S48pGvqLFwtGTtfPBL', function=Function(arguments='{"city":"Oslo"}', name='get_weather'), type='function')])

In [14]:
# Check messages structure
messages.append(ai_message)
messages

[SystemMessage(content="You are a helpful assistant that can access a tool to get current temperature for cities. Use the tool whenever someone asks about the weather or temperature in a specific location. Infor the user if you don't know the answer.", role='system'),
 UserMessage(content='How cold is it in Oslo?', role='user'),
 AIMessage(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(id='call_LdCIo0S48pGvqLFwtGTtfPBL', function=Function(arguments='{"city":"Oslo"}', name='get_weather'), type='function')])]

In [15]:
# Tool call id will be required later when creating the ToolMessage
tool_call_id = messages[-1].tool_calls[0].id
tool_call_id

'call_LdCIo0S48pGvqLFwtGTtfPBL'

In [16]:
# Extract the arguments
args = json.loads(messages[-1].tool_calls[0].function.arguments)
args

{'city': 'Oslo'}

In [17]:
# Execute the tool with the AI's requested parameters
tool_result = get_weather(**args)
tool_result

{'temperature': '-3°C'}

In [18]:
# Create a tool response message
tool_message = ToolMessage(
    content=tool_result["temperature"], 
    tool_call_id=tool_call_id, 
    name="get_weather"
)
tool_message

ToolMessage(content='-3°C', role='tool', tool_call_id='call_LdCIo0S48pGvqLFwtGTtfPBL', name='get_weather')

In [19]:
# Check messages structure
messages.append(tool_message)
messages

[SystemMessage(content="You are a helpful assistant that can access a tool to get current temperature for cities. Use the tool whenever someone asks about the weather or temperature in a specific location. Infor the user if you don't know the answer.", role='system'),
 UserMessage(content='How cold is it in Oslo?', role='user'),
 AIMessage(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(id='call_LdCIo0S48pGvqLFwtGTtfPBL', function=Function(arguments='{"city":"Oslo"}', name='get_weather'), type='function')]),
 ToolMessage(content='-3°C', role='tool', tool_call_id='call_LdCIo0S48pGvqLFwtGTtfPBL', name='get_weather')]

In [20]:
# Let AI formulate final response
ai_message = chat_model_with_tools.invoke(messages)
ai_message

AIMessage(content='The current temperature in Oslo is -3°C.', role='assistant', tool_calls=None)

In [21]:
# Check messages structure
messages.append(ai_message)
messages

[SystemMessage(content="You are a helpful assistant that can access a tool to get current temperature for cities. Use the tool whenever someone asks about the weather or temperature in a specific location. Infor the user if you don't know the answer.", role='system'),
 UserMessage(content='How cold is it in Oslo?', role='user'),
 AIMessage(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(id='call_LdCIo0S48pGvqLFwtGTtfPBL', function=Function(arguments='{"city":"Oslo"}', name='get_weather'), type='function')]),
 ToolMessage(content='-3°C', role='tool', tool_call_id='call_LdCIo0S48pGvqLFwtGTtfPBL', name='get_weather'),
 AIMessage(content='The current temperature in Oslo is -3°C.', role='assistant', tool_calls=None)]